In [ ]:
#DERS 64

In [ ]:
import pandas as pd
import chardet
import matplotlib.pyplot as plt
import re 
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download("stopwords") 
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import numpy as np

In [ ]:
#Eğer farklı bir coding yapısı varsa o coding yapısını nasıl çözeriz
with open("spam.csv","rb") as f:
    sonuc = chardet.detect(f.read())
print(sonuc)

In [ ]:
data = pd.read_csv("spam.csv", encoding = "Windows-1252")
df = data.copy()
df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1,inplace=True)
df

In [ ]:
df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df = df.rename(columns={
    "v1":"Etiket",
    "v2":"Sms"
})
df.head()

In [ ]:
df.groupby("Etiket").count()

In [ ]:
df.describe() #unique = tekrarsız olarak duranlar demek bizim için tekrarlı olmaları kötü sonuçta tahmin etme işleminde
#o bilgiden fazla olduğu için yanlı bir tahmin etme problemiyle karşı karşıya bırakabilir.
#bu yüzden tekrarlayanları silicez
df = df.drop_duplicates()
df.describe()

In [ ]:
df["Karakter Sayısı"] = df["Sms"].apply(len)
df

In [ ]:
df.hist("Karakter Sayısı",by="Etiket",bins=50)
plt.show()

In [ ]:
le = LabelEncoder()
df["Etiket"] = le.fit_transform(df["Etiket"])
df

In [ ]:
#re.sub("[^a-zA-Z]"," ",df["Sms"][0]) = a-z = Küçük harflerin a'dan z'ye dolaş | Büyük harflerin A'dan Z'ye dolaş
#^ Bu dolaştıkların haricinde anlamı katar
#, "" = "" İçinde ne yazarsa bu dolaştıkların haricinde o ile doldur biz boşluk verdik yani rakamlar diğer şeyler gidicek.
mesaj = re.sub("[^a-zA-Z]"," ",df["Sms"][0])
print(df["Sms"][0])
print(mesaj)

In [ ]:
def harfler(cumle):
    #re.compile("[^a-zA-Z]") = Alfabade bulunan değerleri topla ve yer adlı değişkene ata
    yer = re.compile("[^a-zA-Z]")
    return re.sub(yer," ",cumle)
print(harfler("tuy11RTF"))

In [ ]:
print(df["Sms"][0])
print(harfler(df["Sms"][0]))
print(df["Sms"][0].lower())

In [ ]:
durdurma = stopwords.words("english")
print(durdurma)

In [ ]:
#DERS 65

In [ ]:
print(df["Sms"][0])
print(df["Sms"][1])
print(df["Sms"][2])

In [ ]:
durdurma = set(stopwords.words("english"))
def clean(text):
    temiz_kelimeler = []
    text = text.split()
    for i in text:
        if i not in durdurma:
            temiz_kelimeler.append(i)
        else:
            "A"
    return temiz_kelimeler

In [ ]:
spam = []
ham = []
tum_cumleler = []

for i in range(len(df["Sms"].values)):
    r1 = df["Sms"].values[i]
    r2 = df["Etiket"].values[i]
    
    temizcumle = []
    cumleler = harfler(r1)
    cumleler = cumleler.lower()
    
    for kelimeler in clean(cumleler):
        temizcumle.append(kelimeler)
        
        if r2 == 1:
            spam.append(cumleler)
        else:
            ham.append(cumleler)
    tum_cumleler.append(" ".join(temizcumle))
    
df["Yeni Sms"] = tum_cumleler

df.drop(["Sms","Karakter Sayısı"],axis=1,inplace=True)

In [ ]:
print(df["Yeni Sms"][0])
print(df["Yeni Sms"][1])
print(df["Yeni Sms"][2])

In [ ]:
cv = CountVectorizer()
x = cv.fit_transform(df["Yeni Sms"]).toarray()

In [ ]:
y = df["Etiket"]
X = x

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

model = MultinomialNB()
model.fit(X_train,y_train)
tahmin = model.predict(X_test)

acs = accuracy_score(y_test,tahmin)
print(acs*100)

for i in np.arange(0.0,1.1,0.1):
    model = MultinomialNB(alpha=i)
    model.fit(X_train,y_train)
    tahmin = model.predict(X_test)
    skor = accuracy_score(y_test,tahmin)
    print(f"Alfa {round(i,1)} değeri için Skor: {round(skor*100,2)}")